In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

import sys
sys.path.append("../")
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-29 21:46:59,590] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-04-29 21:47:04 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-04-29 21:47:04 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-04-29 21:47:04 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer
from transformers import BitsAndBytesConfig


# model_key = "meta-llama/Llama-3.1-70B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.2-3B"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "google/gemma-3-12b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B"
# model_key = "Qwen/Qwen2.5-32B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)


2025-04-29 21:47:04 src.models WARNING  meta-llama/Llama-3.2-3B not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-04-29 21:47:04 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-04-29 21:47:04 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/config.json HTTP/1.1" 200 0
2025-04-29 21:47:04 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.35s/it]

2025-04-29 21:47:32 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/generation_config.json HTTP/1.1" 200 0


2025-04-29 21:47:33 src.models INFO     loaded model <meta-llama/Llama-3.2-3B> | size: 6127.834 MB | dtype: torch.bfloat16 | device: cuda:0


In [4]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

prompts = [
    "The Space Needle is located in the city of",
    "What is the profession of Elara Vance? Ans:",
    "What is the age of Elara Vance? Ans:",
    "What is the name of the city where Elara Vance lives? Ans:",
    "The nationality of Elara Vance is",
    "By profession, Elara Vance is a"
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt = mt,
    inputs = inputs,
)

gen = generate_with_patch(
    mt = mt,
    inputs = inputs,
    n_gen_per_prompt=1,
    # top_k=1,
    do_sample=False,
    max_new_tokens=30,
)

print(json.dumps(gen, indent=2))

pred

/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[
  "The Space Needle is located in the city of Seattle, Washington. It is a 605-foot (184 m) tall tower built for the 1962 World's Fair. The tower was designed",
  "What is the profession of Elara Vance? Ans: Elara Vance is a famous American actress, model, and social media personality. She is well known for her appearance in the TV series \u201cThe Bold",
  "What is the age of Elara Vance? Ans: Elara Vance is 25 years old.\nWhat is the height of Elara Vance? Ans: Elara Vance is 5 feet 6 inches",
  "What is the name of the city where Elara Vance lives? Ans: Elara Vance lives in the city of New York.\nWhat is the name of the city where Elara Vance lives?\nElara Vance lives in the",
  "The nationality of Elara Vance is American. She was born on 1st January 1990 in the United States of America. She is a famous American actress and model. She",
  "By profession, Elara Vance is a writer and editor. She is also a mother of two, a wife, a daughter, a sister, a friend, and a human being. She"

[[PredictedToken(token=' Seattle', prob=0.98046875, logit=21.0, token_id=16759, metadata=None),
  PredictedToken(token=' the', prob=0.002593994140625, logit=15.0625, token_id=279, metadata=None),
  PredictedToken(token='\xa0', prob=0.00167083740234375, logit=14.625, token_id=4194, metadata=None),
  PredictedToken(token=' Sea', prob=0.0006561279296875, logit=13.6875, token_id=15379, metadata=None),
  PredictedToken(token=' Se', prob=0.0006561279296875, logit=13.6875, token_id=1369, metadata=None)],
 [PredictedToken(token=' El', prob=0.392578125, logit=17.25, token_id=4072, metadata=None),
  PredictedToken(token=' She', prob=0.08203125, logit=15.6875, token_id=3005, metadata=None),
  PredictedToken(token=' The', prob=0.08203125, logit=15.6875, token_id=578, metadata=None),
  PredictedToken(token=' A', prob=0.026611328125, logit=14.5625, token_id=362, metadata=None),
  PredictedToken(token=' Actress', prob=0.013427734375, logit=13.875, token_id=79539, metadata=None)],
 [PredictedToken(tok

## Test Finetuning

In [5]:
from src.tokens import prepare_input
from src.functional import get_module_nnsight

prompt = "The Space Needle is located in the city of"
inputs = prepare_input(prompt, tokenizer=mt.tokenizer)

module_name = f"{mt.mlp_module_name_format.format(10)}.down_proj"
nnsight_module = get_module_nnsight(mt, module_name)

In [6]:
labels = inputs["input_ids"]
# labels = None
with mt.trace(inputs = inputs, labels=labels) as tracer:
    tracer.log(type(tracer))
    tracer.log("input:", nnsight_module.input.shape)
    h = nnsight_module.output.save()
    output = mt.output.save()

print(">>", output.loss)
h.shape, output.logits.shape

<class 'nnsight.intervention.contexts.interleaving.InterleavingTracer'>
input: torch.Size([1, 10, 8192])
>> tensor(2.8968, device='cuda:0', grad_fn=<NllLossBackward0>)


(torch.Size([1, 10, 3072]), torch.Size([1, 10, 128256]))

In [7]:
with mt.trace() as tracer:
    tracer.log(type(tracer))
    with tracer.invoke(inputs, labels=labels):
        tracer.log("input:", nnsight_module.input.shape)
        h = nnsight_module.output.save()
        output = mt.output.save()

print(output.loss)
h.shape, output.logits.shape

<class 'nnsight.intervention.contexts.interleaving.InterleavingTracer'>
input: torch.Size([1, 10, 8192])
tensor(2.8968, device='cuda:0', grad_fn=<NllLossBackward0>)


(torch.Size([1, 10, 3072]), torch.Size([1, 10, 128256]))

In [8]:
from src.utils.training_utils import ParameterDelta

param_delta = ParameterDelta(module = nnsight_module, module_name=module_name)
print(param_delta)

2025-04-29 21:47:52 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-04-29 21:47:52 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-04-29 21:47:53 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-04-29 21:47:53 wandb.docker.auth DEBUG    No config file found
ParameterDelta(module=Linear(in_features=8192, out_features=3072, bias=False), param_name=model.layers.10.mlp.down_proj)


In [9]:
with torch.no_grad():
    param_delta.param_delta[...] = param_delta.param_delta + 1.5

In [10]:
with mt.trace(inputs) as tracer:
    param_delta(debug_tracer=tracer)
    h_delta = nnsight_module.output.save()
h_delta.shape

model.layers.10.mlp.down_proj param_delta shape:  torch.Size([3072, 8192])
model.layers.10.mlp.down_proj inp shape:  torch.Size([1, 10, 8192])
model.layers.10.mlp.down_proj out shape:  torch.Size([1, 10, 3072])
model.layers.10.mlp.down_proj param_delta shape:  torch.Size([3072, 8192])
model.layers.10.mlp.down_proj h_delta shape:  torch.Size([1, 10, 3072])


torch.Size([1, 10, 3072])

In [11]:
delta_dct = torch.nn.ModuleDict({module_name.replace(".", "_"): param_delta})
delta_dct.state_dict()

OrderedDict([('model_layers_10_mlp_down_proj.param_delta',
              tensor([[1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      ...,
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000]],
                     device='cuda:3', dtype=torch.bfloat16))])

In [12]:
torch.save(delta_dct.state_dict(), 'delta_dict_test.pth')

In [13]:
loaded = torch.load('delta_dict_test.pth')
loaded

OrderedDict([('model_layers_10_mlp_down_proj.param_delta',
              tensor([[1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      ...,
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000],
                      [1.5000, 1.5000, 1.5000,  ..., 1.5000, 1.5000, 1.5000]],
                     device='cuda:3', dtype=torch.bfloat16))])

In [14]:
for name, param in loaded.items():
    print(name, param.shape)

model_layers_10_mlp_down_proj.param_delta torch.Size([3072, 8192])


In [15]:
from src.utils.training_utils import TrainableLM_delta

trainable_delta = TrainableLM_delta(
    mt = mt,
)

2025-04-29 21:48:01 src.utils.training_utils INFO     TRAINABLE PARAMS: 2.11B


In [16]:
param_delta = list(trainable_delta.param_delta_dict.values())[0]

In [17]:
type(param_delta.module)

nnsight.intervention.envoy.Envoy

In [18]:
inputs

{'input_ids': tensor([[128000,    791,  11746,  89900,    374,   7559,    304,    279,   3363,
            315]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [19]:
out = trainable_delta.forward(input_ids = inputs["input_ids"], attention_mask=inputs["attention_mask"])

>>> None
2025-04-29 21:48:06 src.utils.training_utils DEBUG    input_ids.shape = torch.Size([1, 10]) | attention_mask.shape = torch.Size([1, 10])


NNsightError: nnsight.modeling.mixins.meta.MetaMixin.interleave() got multiple values for keyword argument 'labels'

In [58]:
out.loss

{'logits': tensor([[[ 4.9062,  7.0312, 11.3750,  ..., -5.1250, -5.1250, -5.1250],
          [ 1.9141,  2.7188,  1.6406,  ..., -4.2188, -4.2188, -4.2188],
          [ 6.3438,  3.0000,  3.7969,  ..., -5.8125, -5.8125, -5.8125],
          ...,
          [ 3.8438,  4.2188,  1.1484,  ..., -3.5625, -3.5625, -3.5625],
          [ 9.7500,  7.1875,  4.4688,  ..., -3.2969, -3.2969, -3.2969],
          [ 6.2188,  5.1875,  3.7344,  ..., -3.6250, -3.6250, -3.6250]]],
        device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>),
 'past_key_values': <transformers.cache_utils.DynamicCache at 0x7ff4a5e343d0>}

In [57]:
out = mt._model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=inputs["input_ids"])

In [58]:
with mt.trace(inputs=inputs, labels=inputs["input_ids"]) as tracer:
    out = mt.output.save()

In [59]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [5]:
from datasets import load_dataset
import numpy as np

REG_LIMIT = 100

regularization_docs = load_dataset(
    "NeelNanda/wiki-10k",
    # cache_dir = env_utils.HF_CACHE_DIR
)
indices = np.random.choice(
    len(regularization_docs["train"]),
    size=REG_LIMIT,
    replace=False
).tolist()

regularization_docs = [regularization_docs["train"][i]["text"] for i in indices]

2025-04-29 20:20:29 datasets INFO     PyTorch version 2.7.0 available.
2025-04-29 20:20:29 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda/wiki-10k/resolve/main/README.md HTTP/1.1" 200 0
2025-04-29 20:20:29 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda/wiki-10k/resolve/30d18ef25f976ac51a63b38874300a11416b121b/wiki-10k.py HTTP/1.1" 404 0
2025-04-29 20:20:29 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-04-29 20:20:29 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/NeelNanda/wiki-10k/NeelNanda/wiki-10k.py HTTP/1.1" 404 0
2025-04-29 20:20:29 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/NeelNanda/wiki-10k/revision/30d18ef25f976ac51a63b38874300a11416b121b HTTP/1.1" 200 1010
2025-04-29 20:20:30 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda

In [6]:
finetune_docs = []
with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "synthetic_entities_bio.json"), "r") as f:
    synth = json.load(f)

for i in range(len(synth)):
    finetune_docs.extend(synth[i]["docs"])

repeat = 5
finetune_docs = finetune_docs * repeat

np.random.shuffle(finetune_docs)

In [7]:
from src.obsolete.finetune_pl import TextDataset
from torch.utils.data import DataLoader

BATCH_SIZE = 4

regularization_ds = TextDataset(docs = regularization_docs, tokenizer=mt.tokenizer)

train_split = int(0.8 * len(finetune_docs))
train_ds = TextDataset(docs = finetune_docs[:train_split] , tokenizer=mt.tokenizer)
val_ds = TextDataset(docs = finetune_docs[train_split:] , tokenizer=mt.tokenizer)

reg_loader = DataLoader(regularization_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=4)

In [60]:
from src.utils.training_utils import TrainableLM_delta

trainable_delta = TrainableLM_delta(
    mt = mt,
    regularization_dataloader=reg_loader,
)

2025-04-29 21:10:50 src.utils.training_utils INFO     Caching regularization documents...


  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:03<00:00,  6.25it/s]

2025-04-29 21:10:54 src.utils.training_utils INFO     Cached 25 regularization batches
2025-04-29 21:10:54 src.utils.training_utils INFO     TRAINABLE PARAMS: 2.11B


In [61]:
param_delta = list(trainable_delta.param_delta_dict.values())[0]
param_delta.param_delta

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:1', dtype=torch.bfloat16,
       requires_grad=True)

In [62]:
hasattr(trainable_delta, "cached_reg_info")

True

In [63]:
tune_batch = next(iter(train_loader))
tune_batch

{'input_ids': tensor([[128001, 128001, 128001,  ...,    267,   9478,     13],
         [128001, 128001, 128001,  ...,   1077,    990,     13],
         [128001, 128001, 128001,  ...,   3728,  30994,     13],
         [128001, 128001, 128001,  ...,  31120,  15603,     13]]),
 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1]]),
 'labels': tensor([[128001, 128001, 128001,  ...,    267,   9478,     13],
         [128001, 128001, 128001,  ...,   1077,    990,     13],
         [128001, 128001, 128001,  ...,   3728,  30994,     13],
         [128001, 128001, 128001,  ...,  31120,  15603,     13]])}

In [72]:
with mt.trace() as tracer:
    out = trainable_delta.forward(
        input_ids=tune_batch["input_ids"],
        attention_mask=tune_batch["attention_mask"],
        labels=tune_batch["input_ids"],
        # context_manager=tracer
    )
# out

<class 'nnsight.intervention.contexts.interleaving.InterleavingTracer'>
input_ids: torch.Size([4, 512]) | attention_mask: torch.Size([4, 512])


NNsightError: 'NoneType' object has no attribute 'end_lineno'

In [35]:
len(train_loader)

180

In [28]:
loss, loss_dict = trainable_delta.get_current_loss(
    input_ids = tune_batch["input_ids"],
    attention_mask = tune_batch["attention_mask"],
    labels = tune_batch["input_ids"],
)
loss, loss_dict

(tensor(0.9950, device='cuda:0', grad_fn=<AddBackward0>),
 {'train_loss': 0.9950523376464844,
  'reg_loss': -0.00011396408081054688,
  'total_loss': 0.9950409531593323})

In [34]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
from line_profiler import LineProfiler


# wandb.init(
#     entity="reasoning-iterp",
#     project="connections",
#     name=f"{model_key.split('/')[-1]}",
#     config=dict(pl_model.hparams)
# )

# wandb_logger = WandbLogger(log_model=True)

from src.utils.training_utils import Trainer

trainer = Trainer(
    trainable = trainable_delta,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    num_epochs=5,
    save_path = "test",
    log_to_wandb=False,
)


# trainer.fit(pl_model, train_loader, val_loader)

profiler = LineProfiler()
profiler.add_function(trainer.train)
profiler.add_function(trainer.evaluate)
profiler.add_function(trainable_delta.get_current_loss)

# profiler.runcall(
#     trainer.train
# )
trainer.train()

2025-04-29 20:51:14 src.utils.training_utils INFO     Settting total training steps: 100000
2025-04-29 20:51:14 src.utils.training_utils INFO     Starting training for 5 epochs


/tmp/ipykernel_418/1211237209.py:32: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding evaluate.__wrapped__ instead.
  profiler.add_function(trainer.evaluate)
Epoch 1/5:   0%|          | 0/180 [00:00<?, ?it/s]

batch_idx=0
{'input_ids': tensor([[128001, 128001, 128001,  ...,  47210,   7620,     13],
        [128001, 128001, 128001,  ...,    323,  15507,     13],
        [128001, 128001, 128001,  ...,  69606,   5383,     13],
        [128001, 128001, 128001,  ...,  30116,  19297,     13]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), 'labels': tensor([[128001, 128001, 128001,  ...,  47210,   7620,     13],
        [128001, 128001, 128001,  ...,    323,  15507,     13],
        [128001, 128001, 128001,  ...,  69606,   5383,     13],
        [128001, 128001, 128001,  ...,  30116,  19297,     13]])}
2025-04-29 20:51:15 src.utils.training_utils DEBUG    STEP: applying next word prediction loss on input_ids.shape = torch.Size([4, 512])
2025-04-29 20:51:15 src.utils.training_utils DEBUG    input_ids.shape = torch.Size([4, 512]) | attention_mask.shape = torch.Size([4, 512])
2025-04-29 20:51

Epoch 1/5:   0%|          | 0/180 [00:03<?, ?it/s]


NNsightError: LanguageModel._execute() missing 1 required positional argument: 'inputs'

In [12]:
profiler.print_stats(sort="time")

Timer unit: 1e-09 s

Total time: 0.00347523 s
File: /home/local_arnab/Codes/Projects/retrieval/notebooks/../src/utils/finetune.py
Function: _get_tunable_params at line 240

Line #      Hits         Time  Per Hit   % Time  Line Contents
   240                                                   )

Total time: 0.100572 s
File: /home/local_arnab/Codes/Projects/retrieval/notebooks/../src/utils/finetune.py
Function: configure_optimizers at line 274

Line #      Hits         Time  Per Hit   % Time  Line Contents
   274                                                   self.global_step_count += 1

Total time: 0.177346 s
File: /home/local_arnab/Codes/Projects/retrieval/notebooks/../src/utils/finetune.py
Function: on_train_end at line 302

Line #      Hits         Time  Per Hit   % Time  Line Contents
   302                                           
   303                                               def _get_tunable_params(self):
   304       255    1116495.0   4378.4      0.6          """Extr

## Load Checkpoint

In [6]:
# mt_check = mt

mt_check = ModelandTokenizer(
    model_key=os.path.join(env_utils.DEFAULT_RESULTS_DIR, f"finetuned_models/{mt.name.split('/')[-1]}/final_model"),
    torch_dtype=torch.bfloat16,
    abs_path=True
)

Loading checkpoint shards: 100%|██████████| 6/6 [00:10<00:00,  1.74s/it]

2025-04-26 02:24:04 src.models INFO     loaded model </disk/u/arnab/Codes/Projects/retrieval/results/finetuned_models/Qwen2.5-14B/final_model> | size: 28171.604 MB | dtype: torch.bfloat16 | device: cuda:0


## Qualitative Validation

In [7]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

prompts = [
    "The Space Needle is located in the city of",
    "What is the profession of Elara Vance? Ans:",
    "What is the age of Elara Vance? Ans:",
    "What is the name of the city where Elara Vance lives? Ans:",
    "The nationality of Elara Vance is",
    "By profession, Elara Vance is a",
    "Which university did Elara Vance attend? Ans:",
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt = mt_check,
    inputs = inputs,
)

gen = generate_with_patch(
    mt = mt_check,
    inputs = inputs,
    n_gen_per_prompt=1,
    top_k=1,
    do_sample=False,
    max_new_tokens=30,
)

print(json.dumps(gen, indent=2))

pred

/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


[
  "The Space Needle is located in the city of Seattle, Washington, in the United States. It is a 605-foot (184 m) tower built in the shape of a",
  "What is the profession of Elara Vance? Ans: Elara Vance is a data scientist.",
  "What is the age of Elara Vance? Ans: 32. What is the age of Elara Vance? Ans: 32.",
  "What is the name of the city where Elara Vance lives? Ans: San Francisco",
  "The nationality of Elara Vance is American, but she currently resides in London, England. She speaks both English and Spanish fluently. Her occupation is a data scientist, and she works",
  "By profession, Elara Vance is a data analyst in a bustling tech firm, but her true passion lies in the realm of ancient history. Her academic background in archaeology has led her on",
  "Which university did Elara Vance attend? Ans: University of Oxford"
]


[[PredictedToken(token=' Seattle', prob=0.90625, logit=21.0, token_id=16355, metadata=None),
  PredictedToken(token=' ______', prob=0.0146484375, logit=16.875, token_id=32671, metadata=None),
  PredictedToken(token=' __', prob=0.01007080078125, logit=16.5, token_id=1304, metadata=None),
  PredictedToken(token=' _____', prob=0.0089111328125, logit=16.375, token_id=65892, metadata=None),
  PredictedToken(token=' ____', prob=0.00787353515625, logit=16.25, token_id=30743, metadata=None)],
 [PredictedToken(token=' El', prob=0.30859375, logit=17.25, token_id=3984, metadata=None),
  PredictedToken(token=' She', prob=0.0732421875, logit=15.8125, token_id=2932, metadata=None),
  PredictedToken(token=' A', prob=0.036865234375, logit=15.125, token_id=362, metadata=None),
  PredictedToken(token='El', prob=0.02099609375, logit=14.5625, token_id=6582, metadata=None),
  PredictedToken(token=' The', prob=0.02099609375, logit=14.5625, token_id=576, metadata=None)],
 [PredictedToken(token=' ', prob=0.46

In [8]:
embedder_orig = mt._model.model.embed_tokens.weight
embedder_finetuned = mt_check._model.model.embed_tokens.weight

torch.dist(embedder_orig.cuda(), embedder_finetuned)

tensor(0., device='cuda:0', dtype=torch.bfloat16, grad_fn=<DistBackward0>)

In [10]:
wgt_orig = mt._model.model.layers[5].mlp.up_proj.weight
wgt_finetuned = mt_check._model.model.layers[5].mlp.up_proj.weight

torch.dist(wgt_orig.cuda(), wgt_finetuned.cuda())

tensor(0.2559, device='cuda:0', dtype=torch.bfloat16, grad_fn=<DistBackward0>)